In [1]:
import numpy as np
# load text data

txt_data = "abcdefghijklmnopqrstuvwxyz abcdefghijklmnopqrstuvwxyz abcdefghijklmnopqrstuvwxyz " # input data
# txt_data = open('input.txt', 'r').read() # test external files

chars = list(set(txt_data)) # split and remove duplicate characters. convert to list.

num_chars = len(chars) # the number of unique characters
txt_data_size = len(txt_data)

print("unique characters : ", num_chars) # You can see the number of unique characters in your input data.
print("txt_data_size : ", txt_data_size)

unique characters :  27
txt_data_size :  81


In [2]:
# one hot encode
char_to_int = dict((c, i) for i, c in enumerate(chars)) # "enumerate" retruns index and value. Convert it to dictionary
int_to_char = dict((i, c) for i, c in enumerate(chars))
print(char_to_int)
print("----------------------------------------------------")
print(int_to_char)
print("----------------------------------------------------")
# integer encode input data
integer_encoded = [char_to_int[i] for i in txt_data] # "integer_encoded" is a list which has a sequence converted from an original data to integers.
print(integer_encoded)
print("----------------------------------------------------")
print("data length : ", len(integer_encoded))

{'x': 0, 'b': 1, 'z': 2, 'k': 3, 'q': 4, 'u': 5, 'p': 6, 'j': 7, 'n': 8, 'm': 9, 'y': 10, 'e': 11, 'a': 12, ' ': 13, 'r': 14, 'i': 15, 'd': 16, 'c': 17, 's': 18, 'l': 19, 'h': 20, 'g': 21, 'f': 22, 'v': 23, 't': 24, 'w': 25, 'o': 26}
----------------------------------------------------
{0: 'x', 1: 'b', 2: 'z', 3: 'k', 4: 'q', 5: 'u', 6: 'p', 7: 'j', 8: 'n', 9: 'm', 10: 'y', 11: 'e', 12: 'a', 13: ' ', 14: 'r', 15: 'i', 16: 'd', 17: 'c', 18: 's', 19: 'l', 20: 'h', 21: 'g', 22: 'f', 23: 'v', 24: 't', 25: 'w', 26: 'o'}
----------------------------------------------------
[12, 1, 17, 16, 11, 22, 21, 20, 15, 7, 3, 19, 9, 8, 26, 6, 4, 14, 18, 24, 5, 23, 25, 0, 10, 2, 13, 12, 1, 17, 16, 11, 22, 21, 20, 15, 7, 3, 19, 9, 8, 26, 6, 4, 14, 18, 24, 5, 23, 25, 0, 10, 2, 13, 12, 1, 17, 16, 11, 22, 21, 20, 15, 7, 3, 19, 9, 8, 26, 6, 4, 14, 18, 24, 5, 23, 25, 0, 10, 2, 13]
----------------------------------------------------
data length :  81


In [3]:
# Not actually used.

onehot_encoded = []

for ix in integer_encoded: # ix is an index mapped to a unique character.
    letter = [0 for _ in range(len(chars))] # A list len is equal to the number of unique characters and whose elements are all zero.
    letter[ix] = 1 # 'letter' is a one-hot vector.
    onehot_encoded.append(letter) # Add a 1d list(a vector for one character).
onehot_encoded = np.array(onehot_encoded) # list to np-array   

print(onehot_encoded.shape)     #  = (len(data),len(chars))
print(onehot_encoded)

# invert encoding
inverted = int_to_char[np.argmax(onehot_encoded[0])] # "argmax" returns the index of the largest value. 
print(inverted)

(81, 27)
[[0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
a


## Initialization

In [4]:
# hyperparameters

iteration = 5000
sequence_length = 10
batch_size = round((txt_data_size /sequence_length)+0.5) # = math.ceil
hidden_size = 100  # size of hidden layer of neurons.  
learning_rate = 1e-1

# model parameters

W_xh = np.random.randn(hidden_size, num_chars)*0.01     # weight input -> hidden. 
W_hh = np.random.randn(hidden_size, hidden_size)*0.01   # weight hidden -> hidden
W_hy = np.random.randn(num_chars, hidden_size)*0.01     # weight hidden -> output

b_h = np.zeros((hidden_size, 1)) # hidden bias
b_y = np.zeros((num_chars, 1)) # output bias

h_prev = np.zeros((hidden_size,1)) # h_(t-1)

## Forward Propagation

In [5]:
def forwardprop(inputs, targets, h_prev):
    # Since the RNN receives the sequence, the weights are not updated during one sequence.
    xs, hs, ys, ps = {}, {}, {}, {} # dictionary
    hs[-1] = np.copy(h_prev) # Copy previous hidden state vector to -1 key value.
    loss = 0 # loss initialization
    
    for t in range(len(inputs)): # t is a "time step" and is used as a key(dic).  
        xs[t] = np.zeros((num_chars,1)) 
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(W_xh, xs[t]) + np.dot(W_hh, hs[t-1]) + b_h) # hidden state. 
        ys[t] = np.dot(W_hy, hs[t]) + b_y # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars. 
        # Softmax. -> The sum of probabilities is 1 even without the exp() function, but all of the elements are positive through the exp() function.
 
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss). Efficient and simple code

#         y_class = np.zeros((num_chars, 1)) 
#         y_class[targets[t]] =1
#         loss += np.sum(y_class*(-np.log(ps[t]))) # softmax (cross-entropy loss)        

    return loss, ps, hs, xs 

## Backward Propagation

In [6]:
def backprop(ps, inputs, hs, xs):

    dWxh, dWhh, dWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy) # make all zero matrices.
    dbh, dby = np.zeros_like(b_h), np.zeros_like(b_y)
    dhnext = np.zeros_like(hs[0]) # (hidden_size,1) 

    # reversed
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t]) # shape (num_chars,1).  "dy" means "dloss/dy"
        dy[targets[t]] -= 1 # backprop into y. After taking the soft max in the input vector, subtract 1 from the value of the element corresponding to the correct label.
        dWhy += np.dot(dy, hs[t].T)
        dby += dy 
        dh = np.dot(W_hy.T, dy) + dhnext # backprop into h. 
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity #tanh'(x) = 1-tanh^2(x)
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(W_hh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]: 
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients.  
    
    return dWxh, dWhh, dWhy, dbh, dby

## Training

In [7]:
ata_pointer = 0

# memory variables for Adagrad
mWxh, mWhh, mWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy)
mbh, mby = np.zeros_like(b_h), np.zeros_like(b_y) 

for i in range(iteration):
    h_prev = np.zeros((hidden_size,1)) # reset RNN memory
    data_pointer = 0 # go from start of data
    
    for b in range(batch_size):
        inputs = [char_to_int[ch] for ch in txt_data[data_pointer:data_pointer+sequence_length]]
        targets = [char_to_int[ch] for ch in txt_data[data_pointer+1:data_pointer+sequence_length+1]] # t+1        
            
        if (data_pointer+sequence_length+1 >= len(txt_data) and b == batch_size-1): # processing of the last part of the input data. 
#             targets.append(char_to_int[txt_data[0]])   # When the data doesn't fit, add the first char to the back.
            targets.append(char_to_int[" "])   # When the data doesn't fit, add space(" ") to the back.

        # forward
        loss, ps, hs, xs = forwardprop(inputs, targets, h_prev)
#         print(loss)
        # backward
        dWxh, dWhh, dWhy, dbh, dby = backprop(ps, inputs, hs, xs) 
        
    # perform parameter update with Adagrad
        for param, dparam, mem in zip([W_xh, W_hh, W_hy, b_h, b_y], 
                                    [dWxh, dWhh, dWhy, dbh, dby], 
                                    [mWxh, mWhh, mWhy, mbh, mby]):
            mem += dparam * dparam # elementwise
            param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update      
        data_pointer += sequence_length # move data pointer
        
    if i % 100 == 0:
        print ('iter %d, loss: %f' % (i, loss)) # print progress

iter 0, loss: 3.814769
iter 100, loss: 0.015266
iter 200, loss: 0.006316
iter 300, loss: 0.003864
iter 400, loss: 0.002754
iter 500, loss: 0.002123
iter 600, loss: 0.001722
iter 700, loss: 0.001446
iter 800, loss: 0.001246
iter 900, loss: 0.001093
iter 1000, loss: 0.000974
iter 1100, loss: 0.000878
iter 1200, loss: 0.000800
iter 1300, loss: 0.000735
iter 1400, loss: 0.000679
iter 1500, loss: 0.000632
iter 1600, loss: 0.000591
iter 1700, loss: 0.000555
iter 1800, loss: 0.000523
iter 1900, loss: 0.000495
iter 2000, loss: 0.000470
iter 2100, loss: 0.000447
iter 2200, loss: 0.000427
iter 2300, loss: 0.000408
iter 2400, loss: 0.000391
iter 2500, loss: 0.000376
iter 2600, loss: 0.000361
iter 2700, loss: 0.000348
iter 2800, loss: 0.000335
iter 2900, loss: 0.000324
iter 3000, loss: 0.000313
iter 3100, loss: 0.000303
iter 3200, loss: 0.000293
iter 3300, loss: 0.000284
iter 3400, loss: 0.000276
iter 3500, loss: 0.000268
iter 3600, loss: 0.000260
iter 3700, loss: 0.000253
iter 3800, loss: 0.00024

## Prediction

In [8]:
def predict(test_char, length):
    x = np.zeros((num_chars, 1)) 
    x[char_to_int[test_char]] = 1
    ixes = []
    h = np.zeros((hidden_size,1))

    for t in range(length):
        h = np.tanh(np.dot(W_xh, x) + np.dot(W_hh, h) + b_h) 
        y = np.dot(W_hy, h) + b_y
        p = np.exp(y) / np.sum(np.exp(y)) 
        ix = np.random.choice(range(num_chars), p=p.ravel()) # ravel -> rank0
        # "ix" is a list of indexes selected according to the soft max probability.
        x = np.zeros((num_chars, 1)) # init
        x[ix] = 1 
        ixes.append(ix) # list
    txt = ''.join(int_to_char[i] for i in ixes)
    print ('----\n %s \n----' % (txt, ))

In [9]:
predict('a', 30)

----
 bcdefghijklmnopqrstuvwxyz abcd 
----


In [10]:
predict('b', 30)

----
 cdexyz abcdefghijklmnopqrstuvw 
----
